In [3]:
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score, mean_squared_error

class KNN:
    def __init__(self, k=5, mode='classification'):
        """
        Initialize K-NN model.
        
        Parameters:
        - k: Number of neighbors to consider (default: 5)
        - mode: 'classification' or 'regression' (default: 'classification')
        """
        self.k = k
        self.mode = mode
        self.X_train = None
        self.y_train = None
        
    def fit(self, X_train, y_train):
        """
        Store the training data.
        
        Parameters:
        - X_train: Feature matrix (n_samples, n_features)
        - y_train: Target values (n_samples,)
        """
        self.X_train = np.array(X_train)
        self.y_train = np.array(y_train)
        
    def _euclidean_distance(self, x1, x2):
        """
        Compute Euclidean distance between two points.
        """
        return np.sqrt(np.sum((x1 - x2) ** 2))
    
    def _get_neighbors(self, x):
        """
        Find the k-nearest neighbors of a point x.
        """
        distances = []
        for i, x_train in enumerate(self.X_train):
            dist = self._euclidean_distance(x, x_train)
            distances.append((i, dist))
        
        # Sort distances and select top k neighbors
        distances.sort(key=lambda x: x[1])
        neighbors = distances[:self.k]
        return neighbors
    
    def predict(self, X_test):
        """
        Make predictions for test data.
        
        Parameters:
        - X_test: Feature matrix (n_samples, n_features)
        
        Returns:
        - Predictions (n_samples,)
        """
        X_test = np.array(X_test)
        predictions = []
        
        for x in X_test:
            # Get k-nearest neighbors
            neighbors = self._get_neighbors(x)
            neighbor_indices = [idx for idx, _ in neighbors]
            neighbor_labels = self.y_train[neighbor_indices]
            
            if self.mode == 'classification':
                # Majority voting for classification
                majority_vote = Counter(neighbor_labels).most_common(1)[0][0]
                predictions.append(majority_vote)
            elif self.mode == 'regression':
                # Mean aggregation for regression
                mean_value = np.mean(neighbor_labels)
                predictions.append(mean_value)
            else:
                raise ValueError("Mode must be 'classification' or 'regression'")
        
        return np.array(predictions)
    
    def score(self, X_test, y_test):
        """
        Evaluate the model on test data.
        
        Parameters:
        - X_test: Feature matrix (n_samples, n_features)
        - y_test: True target values (n_samples,)
        
        Returns:
        - Accuracy (for classification) or MSE (for regression)
        """
        y_pred = self.predict(X_test)
        if self.mode == 'classification':
            return accuracy_score(y_test, y_pred)
        elif self.mode == 'regression':
            return mean_squared_error(y_test, y_pred)
        else:
            raise ValueError("Mode must be 'classification' or 'regression'")


In [4]:
# Example usage
if __name__ == "__main__":
    # Classification example
    print("K-NN Classification Example")
    print("===========================")
    X_train = [[1, 2], [2, 3], [3, 3], [6, 7], [7, 8], [8, 8]]
    y_train = [0, 0, 0, 1, 1, 1]
    
    knn_classifier = KNN(k=3, mode='classification')
    knn_classifier.fit(X_train, y_train)
    
    X_test = [[2, 2], [7, 7]]
    y_test = [0, 1]
    predictions = knn_classifier.predict(X_test)
    accuracy = knn_classifier.score(X_test, y_test)
    
    print(f"Predictions: {predictions}")
    print(f"Accuracy: {accuracy:.2f}")
    
    # Regression example
    print("\nK-NN Regression Example")
    print("=======================")
    X_train = [[1], [2], [3], [6], [7], [8]]
    y_train = [2, 4, 6, 12, 14, 16]
    
    knn_regressor = KNN(k=3, mode='regression')
    knn_regressor.fit(X_train, y_train)
    
    X_test = [[2], [7]]
    y_test = [4, 14]
    predictions = knn_regressor.predict(X_test)
    mse = knn_regressor.score(X_test, y_test)
    
    print(f"Predictions: {predictions}")
    print(f"MSE: {mse:.2f}")

K-NN Classification Example
Predictions: [0 1]
Accuracy: 1.00

K-NN Regression Example
Predictions: [ 4. 14.]
MSE: 0.00
